<a href="https://colab.research.google.com/github/kor1999/Sentence-Similarity-LLM-course-/blob/main/Sentence_similarity_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install sentence-transformers

In [ ]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation, util
from torch.utils.data import DataLoader

In [ ]:
# Loading dataset of STS Benchmark
dataset = load_dataset('stsb_multi_mt', 'en')

# print info about dataset
print(dataset)

# Dividing dataset to train, validation and test
train_data = dataset['train']
val_data = dataset['dev']
test_data = dataset['test']

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 5749
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 1379
    })
    dev: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 1500
    })
})


In [ ]:
# print examples of train data
print(train_data[:2])

{'sentence1': ['A plane is taking off.', 'A man is playing a large flute.'], 'sentence2': ['An air plane is taking off.', 'A man is playing a flute.'], 'similarity_score': [5.0, 3.799999952316284]}


In [ ]:
# Преобразование данных в InputExample
train_examples = [InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score']) for row in train_data]
val_examples = [InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score']) for row in val_data]
test_examples = [InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score']) for row in test_data]


In [ ]:
# Создание DataLoader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
val_dataloader = DataLoader(val_examples, shuffle=False, batch_size=16)
test_dataloader = DataLoader(test_examples, shuffle=False, batch_size=16)

In [ ]:
# Создание DataLoader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
val_dataloader = DataLoader(val_examples, shuffle=False, batch_size=16)
test_dataloader = DataLoader(test_examples, shuffle=False, batch_size=16)

# Загрузка предобученной модели
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
# Настройка функции потерь
train_loss = losses.CosineSimilarityLoss(model)

In [ ]:
# Обучение модели
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=1,
          warmup_steps=100,
          evaluator=None,
          evaluation_steps=0,
          output_path=None)


Step,Training Loss


In [ ]:
# Оценка модели на валидационных данных
evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(val_examples, name='sts-dev')
results = model.evaluate(evaluator)
print("Pearson correlation (cosine similarity):", results['sts-dev_pearson_cosine'])

#model.evaluate(val_dataloader)


Pearson correlation (cosine similarity): 0.676102017562181


In [ ]:
# Функция для вычисления косинусного сходства между двумя словами
def check_similarity(word1, word2, model):
    # Получение эмбеддингов для двух слов
    embedding1 = model.encode(word1, convert_to_tensor=True)
    embedding2 = model.encode(word2, convert_to_tensor=True)

    # Вычисление косинусного сходства
    cosine_similarity = util.pytorch_cos_sim(embedding1, embedding2)

    return cosine_similarity.item()

In [ ]:
# Пример слов для проверки

word1 = "cat"
word2 = "dog"
word3 = "table"
word4 = "tank"
word5 = "cats"
# Вычисление и вывод схожести

print(f"Cosine similarity between '{word1}' and '{word2}': {check_similarity(word1, word2, model)}")
print(f"Cosine similarity between '{word1}' and '{word3}': {check_similarity(word1, word3, model)}")
print(f"Cosine similarity between '{word1}' and '{word4}': {check_similarity(word1, word4, model)}")
print(f"Cosine similarity between '{word1}' and '{word5}': {check_similarity(word1, word5, model)}")
print(f"Cosine similarity between '{word1}' and '{word1}': {check_similarity(word1, word1, model)}")

Cosine similarity between 'cat' and 'dog': 0.9970299601554871
Cosine similarity between 'cat' and 'table': 0.9760968685150146
Cosine similarity between 'cat' and 'tank': 0.9943073987960815
Cosine similarity between 'cat' and 'cats': 0.9988530874252319
Cosine similarity between 'cat' and 'cat': 0.9999998807907104
